# Password cracking using multithreading

In [ ]:
!apt-get update
!apt-get install -y libxcrypt-dev


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,308 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,789 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-up

In [ ]:
%%writefile EncryptSHA512.c

#include <string.h>
#include <stdio.h>
#include <stdlib.h>
#include <crypt.h>
#include <unistd.h>

#define SALT "$6$AS$"

int main(int argc, char *argv[]){
    if (argc != 2) {
        printf("Usage: %s <plain_password>\n", argv[0]);
        return 1;
    }

    printf("Password: %s\n", argv[1]);

    // Encrypt using the specified salt
    char *encrypted = crypt(argv[1], SALT);

    if (encrypted != NULL) {
        printf("Encrypted password: %s\n", encrypted);
    } else {
        printf("Error encrypting password.\n");
    }

    return 0;
}


Writing EncryptSHA512.c


In [ ]:
%%writefile CrackAZ99_MT.c

#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <pthread.h>
#include <crypt.h>
#include <unistd.h>

int count = 0; // Counter used to track the number of combinations explored

pthread_mutex_t crypt_mutex = PTHREAD_MUTEX_INITIALIZER;

// Structure for passing data to each thread
typedef struct {
    int start_letter;
    int end_letter;
    char salt[7];
    char *salt_and_encrypted;
} ThreadArgs;

pthread_mutex_t count_mutex = PTHREAD_MUTEX_INITIALIZER;
pthread_mutex_t found_mutex = PTHREAD_MUTEX_INITIALIZER;
int password_found = 0;

// Utility function to copy a substring
void substr(char *dest, char *src, int start, int length) {
    memcpy(dest, src + start, length);
    *(dest + length) = '\0';
}

// Thread function to crack a password
void* crack(void* args) {
    ThreadArgs* thread_args = (ThreadArgs*)args;
    char plain[7]; // Combination currently being checked
    char *enc;     // Pointer to encrypted password
    int x, y, z;

    // Loop through the first letter range assigned to this thread
    for (x = thread_args->start_letter; x <= thread_args->end_letter && !password_found; x++) {
        for (y = 'A'; y <= 'Z' && !password_found; y++) {
            for (z = 0; z <= 99 && !password_found; z++) {
                sprintf(plain, "%c%c%02d", x, y, z);

                pthread_mutex_lock(&crypt_mutex);
                enc = (char *)crypt(plain, thread_args->salt);
                pthread_mutex_unlock(&crypt_mutex);


                // Increment count safely
                pthread_mutex_lock(&count_mutex);
                count++;
                pthread_mutex_unlock(&count_mutex);

                // Check if the current combination matches the given hash
                if (strcmp(thread_args->salt_and_encrypted, enc) == 0) {
                    pthread_mutex_lock(&found_mutex);
                    if (!password_found) {
                        password_found = 1; // Flag to signal that the password is found
                        printf("#%-8d%s %s\n", count, plain, enc);
                    }
                    pthread_mutex_unlock(&found_mutex);
                    return NULL; // Return immediately to stop further searches
                }
            }
        }
    }
    return NULL;
}

// Main function to set up threads and crack the password
int main(int argc, char *argv[]) {
    if (argc != 3) {
        printf("Usage: %s <encrypted_password> <num_threads>\n", argv[0]);
        return 1;
    }

    char *salt_and_encrypted = argv[1];
    int num_threads = atoi(argv[2]);

    if (num_threads <= 0 || num_threads > 26) {
        printf("Number of threads must be between 1 and 26.\n");
        return 1;
    }

    char salt[7];
    substr(salt, salt_and_encrypted, 0, 6);

    // Print the salt being used for debugging
    printf("Using Salt: %s\n", salt);

    // Create threads and assign letter ranges
    pthread_t threads[num_threads];
    ThreadArgs args[num_threads];

    int letters_per_thread = 26 / num_threads;
    int remaining_letters = 26 % num_threads;
    char current_letter = 'A';

    for (int i = 0; i < num_threads; ++i) {
        args[i].start_letter = current_letter;
        args[i].end_letter = current_letter + letters_per_thread - 1;
        if (i < remaining_letters) {
            args[i].end_letter += 1;
        }
        current_letter = args[i].end_letter + 1;

        strcpy(args[i].salt, salt);
        args[i].salt_and_encrypted = salt_and_encrypted;

        pthread_create(&threads[i], NULL, crack, &args[i]);
    }

    // Join threads
    for (int i = 0; i < num_threads; ++i) {
        pthread_join(threads[i], NULL);
    }

    printf("%d solutions explored\n", count);
    return 0;
}



Writing CrackAZ99_MT.c


In [ ]:
!gcc -o EncryptSHA512 EncryptSHA512.c -lcrypt

In [ ]:
!./EncryptSHA512 "AB12"

Password: AB12
Encrypted password: $6$AS$EquwSMfZH6UigdniioE8VWG9qfQ/iburie8TclTB4HCYRomJtmDsM31EqQEbs5Zk09UzWMOtHoXFFmdKRKVoy/


In [ ]:
!gcc -pthread -o CrackAZ99_MT CrackAZ99_MT.c -lcrypt


In [ ]:
!./CrackAZ99_MT '$6$AS$EquwSMfZH6UigdniioE8VWG9qfQ/iburie8TclTB4HCYRomJtmDsM31EqQEbs5Zk09UzWMOtHoXFFmdKRKVoy/' 4


Using Salt: $6$AS$
#617     AB12 $6$AS$EquwSMfZH6UigdniioE8VWG9qfQ/iburie8TclTB4HCYRomJtmDsM31EqQEbs5Zk09UzWMOtHoXFFmdKRKVoy/
620 solutions explored
